**this file contains functions used to generate new schedules when add/remove routes**

1. get_trips_for_routes
2. filter_stoptimes_by_routes

In [48]:
import pandas as pd

In [3]:
def get_trips_for_routes(routes = [30,34,36]):
    ### GET TRIPS FOR CERTAIN ROUTES ###
    # para: routes in list (float number)
    # return: trip_id mapping with rotue numbers in dictionary
    
    trips_dict = dict() # A dictionary mapping routes with multiple trips
    
    # read trips.txt file, keep only 2 columns of routes_short_name and trip_id
    trips = (pd.read_csv(r"mmt_gtfs/trips.txt", sep = ",", engine = "python")
            [['route_short_name','trip_id']])
    
    # for each routes (30,34,36), select their trips and update into stop_dict
    for r in routes:
        trip_id = trips.loc[trips['route_short_name'] == r,:]
        tp = trip_id["trip_id"].tolist()
        
        trips_dict.update({r:tp})

    return(trips_dict)

In [54]:
def filter_stoptimes_by_routes(routes = [30.,34,36]):
    ### READ stop_times.txt, AND FIILTER WITH CERTIAN ROUTES ###
    # para: routes in a list
    # return: filtered stop_times in pd.dataframe
    # calling function: get_trips_for_routes
    
    # get trips into a list
    tr_dict = get_trips_for_routes(routes)
    
    tr_list = []
    for r,tr in tr_dict.items():
        tr_list += tr
    
    # read stoptimes.txt file
    stop_times = pd.read_csv(r"mmt_gtfs/stop_times.txt", sep = ",", engine = "python")
    # filter by trips under certain routes
    stop_times_filter = stop_times.loc[stop_times['trip_id'].isin(tr_list)].reset_index()

    # return filtered stop_times in dataframe    
    return stop_times_filter
    

In [33]:
def convert_time_to_sec(df = stop_times):
    ###Convert str arrival_time/departure_time to int seconds###
    # HH:MM:SS -> HH*60*60+MM*60+SS
    # para: stop_times (a standard GTFS stop_times file)(pd dataframe)
    # return: stop_times with two columns added on
    
    # split string time into hours, minutes, and seconds
    df[['ar_h','ar_m','ar_s']] = df.arrival_time.str.split(':',expand=True)
    df[['dp_h','dp_m','dp_s']] = df.departure_time.str.split(':',expand=True)

    # compute time into seconds
    df=(df.assign(arrival_time = lambda x: df['ar_h'].astype(int)*60*60+df['ar_m'].astype(int)*60+df['ar_s'].astype(int),
                departure_time = lambda x: df['dp_h'].astype(int)*60*60+df['dp_m'].astype(int)*60+df['dp_s'].astype(int))
        .filter(regex = '^((?!ar_).)*$')  # remove intermediate variables
        .filter(regex = '^((?!dp_).)*$'))
    

    return df

In [55]:
stop_times = convert_time_to_sec(filter_stoptimes_by_routes())
stop_times.head()

,index,trip_id,stop_sequence,stop_id,pickup_type,drop_off_type,arrival_time,departure_time,timepoint,stop_headsign,shape_dist_traveled
0,36696,908292,1,7100,0,1,22500,22500,1,E TOWNE: VIA NAKOOSA,0.0092
1,36697,908292,2,7169,0,0,22544,22544,0,E TOWNE: VIA NAKOOSA,0.2630
2,36698,908292,3,9629,0,0,22776,22776,0,E TOWNE: VIA NAKOOSA,1.5965
3,36699,908292,4,9753,0,0,22800,22800,1,E TOWNE: VIA NAKOOSA,1.7287
4,36700,908292,5,9388,0,0,22822,22822,0,E TOWNE: VIA NAKOOSA,1.8701


In [76]:
def get_travel_time(stop_list = [], stop_times = stop_times):
    ### Based on the existing schedule, get the travel time and distance for sequenced two stops, return in a pd dataframe###
    # para: stop_list (a list of existing stop_id)
    #       stop_times (an existing schedule, arrival_times MUST be converted to seconds)
    # return: stop_seq (pd dataframe)
    
    # Create a new df to store outputs
    stop_seq = pd.DataFrame({"upstop_id" : [],
                             "downstop_id":[],
                             "travel_time" : [],
                             "travel_dist" : []})
    
    # for every two sequenced stop_ids in the given stop_list, search for the matching result in stop_times
    for i in range(len(stop_list)-1):
        # get a pair of stop_ids 
        upstop = stop_list[i]
        downstop = stop_list[i+1]

        # search in the existing schedule
        for j in range(len(stop_times)-1):
            if stop_times['stop_id'][j] == upstop:
                if stop_times['stop_id'][j+1] == downstop:
                    if stop_times['stop_sequence'][j+1]-stop_times['stop_sequence'][j] == 1:
                        
                        # get travel time and distance
                        time = stop_times['arrival_time'][j+1]-stop_times['arrival_time'][j]
                        dist = stop_times['shape_dist_traveled'][j+1]-stop_times['shape_dist_traveled'][j]
                        
                        # append to stop_seq the dataframe 
                        row={'upstop_id':upstop,'downstop_id':downstop,'travel_time':time,'travel_dist':dist}
                        stop_seq.loc[i]=row

                        # for every pair (upstop,downstop), once we got a result, move to the next pair
                        break

    return stop_seq



In [78]:
stop_list_test = [9237,9175,9373,9918,9364,9100,9642,9378,9564,9802,9288,1420,1751,1309,1105,7100,7169,7167,7209,7633,9167,9811,9277,9243,9439,9137,9745,9951,9645,9291,9432,9719,9677,9800,9783,9391,9351,9140,9310,9620,9863,9917,9928,9422,9800]
stop_seq = get_travel_time(stop_list_test,stop_times)
stop_seq

,upstop_id,downstop_id,travel_time,travel_dist
0,9237.0,9175.0,33.0,0.1165
1,9175.0,9373.0,68.0,0.2397
2,9373.0,9918.0,43.0,0.1527
3,9918.0,9364.0,70.0,0.2461
4,9364.0,9100.0,86.0,0.2899
5,9100.0,9642.0,63.0,0.2828
6,9642.0,9378.0,53.0,0.2387
7,9378.0,9564.0,21.0,0.0981
8,9564.0,9802.0,23.0,0.1063
9,9802.0,9288.0,57.0,0.2554
